In [1]:
import tensorflow as tf
from ASR_Network import ASR_Network
from DataPipe import DataPipeFactory
from util_function import inform_pooling

/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/p

In [2]:
# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [3]:
ds = DataPipeFactory('../DataFolder/Tensorflow_DataRecord/Student_Answer_Record.tfrecord',
                     '../DataFolder/Siri_Related/Siri_Reference_Sample',
                     '../DataFolder/Siri_Related/Siri_Dense_Index', cache='../DataFolder/cache/datapipe/cached')
# dsp = ds.get_batch_data(10)
# it = iter(dsp)
ds.get_raw_data()

Load cache from ../DataFolder/cache/datapipe/cached


<_LoadDataset element_spec={'ref_mfcc': TensorSpec(shape=(4, None, 80), dtype=tf.float32, name=None), 'MatchSegment': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'valid_ref_word': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'MatchReference': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'valid_ref_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'RecordName': TensorSpec(shape=(), dtype=tf.string, name=None), 'passage_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'valid_ref_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_stu_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_stu_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'stu_mfcc': TensorSpec(shape=(None, 80), dtype=tf.float32, name=None)}>

In [5]:
base_feature = ([None, 80], 512, [128, 256, 384, 512], 3, 2)
base_feature_name = ('input_shape', 'feature_depth', 'channels_list', 'filter_size', 'stack_size')
dense_feature = ([1280], 512, [512, 512], 2)
dense_feature_name = ('input_shape', 'output_shape', 'channels_list', 'stack_size')
word_prediction = ([512], 2500, [512, 1024], 2)
word_prediction_name = ('input_shape', 'output_shape', 'channels_list', 'stack_size')
# Generate dict using zip
# base_feature = dict(zip(base_feature_name, base_feature))
# dense_feature = dict(zip(dense_feature_name, dense_feature))
# word_prediction = dict(zip(word_prediction_name, word_prediction))
base_ratio = 125
hyper_para = {'base_feature': dict(zip(base_feature_name, base_feature)),
              'dense_feature': dict(zip(dense_feature_name, dense_feature)),
              'word_prediction': dict(zip(word_prediction_name, word_prediction)),
              'base_ratio': base_ratio,
              'batch_num': 10}
C = ASR_Network(**hyper_para)

In [6]:
def unpack(d):
    value_s = d['stu_mfcc']
    start_s = tf.RaggedTensor.from_tensor(d['valid_stu_start'], padding=-1.)
    duration_s = tf.RaggedTensor.from_tensor(d['valid_stu_duration'], padding=-1.)

    #unpack with another key ref_mfcc, valid_ref_start, valid_ref_duration
    value_f = d['ref_mfcc']
    start_f = tf.RaggedTensor.from_tensor(d['valid_ref_start'], padding=-1.)
    duration_f = tf.RaggedTensor.from_tensor(d['valid_ref_duration'], padding=-1.)

    # unpack valid_ref_word

    words = tf.RaggedTensor.from_tensor(d['valid_ref_word'], padding=-1)
    return ((value_s, (start_s, duration_s)), (value_f, (start_f, duration_f))), words

In [15]:
dst, dse = ds.k_fold(5,0,10,addition_map=unpack)

In [16]:
it = iter(dst)

In [17]:
data = next(it)

In [18]:
def call(self, inputs, training=False, mask=None):
    audio, (start, duration) = inputs
    # compute the base network
    base_output, maps = self.base_network(audio, training=training)
    # combine base output and maps
    total_maps = [base_output] + maps
    # pooling the total maps
    pooled_maps = self.pooling(total_maps, start, duration)
    # compute the deep feature
    deep_feature = self.deep_feature(pooled_maps, training=training)
    # compute the word prediction
    word_prediction = self.word_prediction(deep_feature, training=training)
    return word_prediction, deep_feature

In [19]:
C(data[0][0])

(<tf.RaggedTensor [[[0.29794404, 0.08054207, 0.7920669, ..., 1.2602772, -0.3411046,
    -0.24284212],
   [0.28876218, -0.26614916, -0.17134641, ..., 1.2873058, 0.07359129,
    2.1470542],
   [-0.4793513, -0.5033953, -1.3805486, ..., 0.29569468, 1.1471448,
    -0.23561561],
   ...,
   [3.7221708, -0.8966097, -0.7043177, ..., 1.9055716, -0.18278909,
    -1.1244692],
   [2.466493, -0.7019958, -0.95875764, ..., 1.3253355, 0.728972, -1.9400762],
   [2.0040302, -0.47695586, -0.8725844, ..., 0.18156624, -0.574592,
    -0.2909061]]                                                             ,
  [[1.425798, 0.3037963, -0.64081126, ..., 0.4296107, -0.92541397,
    0.092755914],
   [1.5341148, -0.2825191, -0.04013071, ..., -0.24971741, -1.0493644,
    0.49545455],
   [1.6560396, -0.040533036, 0.4396667, ..., 0.95623183, -1.7417755,
    0.8066735],
   ...,
   [-0.046067953, -0.5813272, 0.7181413, ..., 0.8827368, -1.6058826,
    1.7906159],
   [0.73624367, 0.116378844, -0.7141137, ..., -0.36123243,

In [12]:
C.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
C.fit(dst, epochs=1)

In [ ]:
nl = tf.keras.layers.BatchNormalization()

In [ ]:
Ragg1 = tf.RaggedTensor.from_row_splits(tf.random.normal([100,80]), row_splits=tf.range(0, 101, 5))
Ragg2 = tf.RaggedTensor.from_row_splits(tf.random.normal([100,80]), row_splits=tf.range(0, 101, 10))

In [ ]:
tf.ragged.map_flat_values(nl, Ragg2)

In [ ]:
class RaggedBatchNormalization(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.batch_norm = tf.keras.layers.BatchNormalization(**kwargs)

    def call(self, inputs, training=None, mask=None):
        # Pass `training` and `mask` arguments to the `call` method of the BatchNormalization layer
        return tf.ragged.map_flat_values(
            lambda x: self.batch_norm(x, training=training, mask=mask), inputs
        )

    def get_config(self):
        return super().get_config()

    @classmethod
    def from_config(cls, config):
        return super().from_config(config)

In [ ]:
rnl = RaggedBatchNormalization()

In [ ]:
rnl(Ragg1)

In [ ]:
nl(tf.random.normal([100,80]))

In [ ]:
Ragg1.ragged_rank

In [ ]:
Ragg1.flat_values

In [21]:
C.evaluate(dse)

     19/Unknown - 63s 3s/step - loss: 117.8727 - word_loss: 113.6317 - deep_loss: 4.2410

KeyboardInterrupt: 